<a href="https://colab.research.google.com/github/Kira1108/langchain-experiments/blob/main/Langchain_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interacting with LLMs, you need memory

In [ ]:
from IPython.display import clear_output

! pip install langchain openai

clear_output()

In [ ]:
import os
import openai
os.environ['OPENBI_API_KEY'] = ""

BaseMemory - 不看就学真是大傻逼       

- memroy_variables: property of variables names
- load_memory_variables: load memroy content and names as dictionary
- save_context: save memory content
- clear: delete all memories



```python
from pydantic import BaseModel
from abc import ABC, abstractmethod

class BaseMemory(BaseModel, ABC):
    """Base interface for memory in chains."""

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    @abstractmethod
    def memory_variables(self) -> List[str]:
        """Input keys this memory class will load dynamically."""

    @abstractmethod
    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Return key-value pairs given the text input to the chain.
        If None, return all memories
        """

    @abstractmethod
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """Save the context of this model run to memory."""

    @abstractmethod
    def clear(self) -> None:
        """Clear memory contents."""
```



### Conversation Buffer Memory

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input": "hi"}, {"ouput": "whats up"})

In [ ]:
memory.memory_variables

['history']

In [ ]:
memory.load_memory_variables({})

{'history': 'Human: hi\nAI: whats up'}

In [ ]:
memory = ConversationBufferMemory()
memory.save_context({"input": "hi"}, {"ouput": "whats up"})

In [ ]:
memory.memory_variables

['history']

In [ ]:
memory.load_memory_variables({})

{'history': 'Human: hi\nAI: whats up'}

In [ ]:
memory.clear()

In [ ]:
memory.load_memory_variables({})

{'history': ''}

Use Memory - Chain organize llms and memories.    
A Chain has a llm and a memory object.   

你去用Memory的时候，你只操心怎么初始化，否则Memory的存在就是垃圾。   

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

# 3.5-turbo is not supported here, it is only supported in chat..
# here we use 3.5 to save tokens
llm = OpenAI(temperature=0, model_name = "gpt-3.5-turbo")
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=memory
)

/usr/local/lib/python3.9/dist-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/langchain/llms/openai.py:608: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [ ]:
conversation.predict(input = "hi there")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi there
AI:

> Finished chain.


'Hello! How can I assist you today?'

In [ ]:
conversation.predict(input = 'are you AI?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi there
AI: Hello! How can I assist you today?
Human: are you AI?
AI:

> Finished chain.


'Yes, I am an AI. Specifically, I am a language model designed to understand and respond to natural language input.'

In [ ]:
memory.load_memory_variables({})

{'history': 'Human: hi there\nAI: Hello! How can I assist you today?\nHuman: are you AI?\nAI: Yes, I am an AI. Specifically, I am a language model designed to understand and respond to natural language input.'}

### ConversationBufferWindowMemory - Just a simple window

In [43]:
from langchain.memory import ConversationBufferWindowMemory

k = 2

memory = ConversationBufferWindowMemory(k = k, return_messages = True)

for i in range(3):
    print(f"\nIteration: {i + 1}")
    memory.save_context({"input":f"Question {i+1}"}, {"output":f"Answer {i+1}"})
    print(memory.load_memory_variables({}))


Iteration: 1
{'history': [HumanMessage(content='Question 1', additional_kwargs={}), AIMessage(content='Answer 1', additional_kwargs={})]}

Iteration: 2
{'history': [HumanMessage(content='Question 1', additional_kwargs={}), AIMessage(content='Answer 1', additional_kwargs={}), HumanMessage(content='Question 2', additional_kwargs={}), AIMessage(content='Answer 2', additional_kwargs={})]}

Iteration: 3
{'history': [HumanMessage(content='Question 2', additional_kwargs={}), AIMessage(content='Answer 2', additional_kwargs={}), HumanMessage(content='Question 3', additional_kwargs={}), AIMessage(content='Answer 3', additional_kwargs={})]}


### ConversationEntityMemory

这里额外收取了实体抽取和实体摘要的钱，除非你钱多，否则不要用，老老实实去网页。     

In [44]:
from langchain.memory import ConversationEntityMemory
llm = OpenAI(temperature=0)

Memory is summarized from context, which also use llms.    

If this cost you too much, please stop using memories.    

In [45]:
memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)

{'history': '', 'entities': {'Deven': '', 'Sam': ''}}

In [46]:
memory.save_context(
    _input,
    {"ouput": " That sounds like a great project! What kind of project are they working on?"}
)
memory.load_memory_variables({'input':"Who is sam?"})

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?',
 'entities': {'Sam': 'Sam is working on a hackathon project with Deven.'}}

In [52]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE, ENTITY_EXTRACTION_PROMPT, ENTITY_SUMMARIZATION_PROMPT
from pydantic import BaseModel
from typing import List, Dict, Any

维护聊天历史的Prompt

In [51]:
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template)

You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in discussions and provide explanations and de

实体抽取的Prompt

In [54]:
print(ENTITY_EXTRACTION_PROMPT.template)

You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.

The conversation history is provided just in case of a coreference (e.g. "What do you know about him" where "him" is defined in a previous line) -- ignore items mentioned there that are not in the last line.

Return the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).

EXAMPLE
Conversation history:
Person #1: how's it going today?
AI: "It's going great! How about you?"
Person #1: good! busy working on Langchain. lots to do.
AI: "That sounds like a lot of work! What kind of things are you doing to make Langchain better?"
Last line:
Person #1: i'm trying to improve Langchain's interfaces, the UX, its integration

实体总结的Prompt，注意这里的实体总结是， FOR EACH ENTITY: DO SUMMARIZE.     
所以你想下实体历史和对话历史，你发送了多少遍请求给openai   

In [55]:
print(ENTITY_SUMMARIZATION_PROMPT.template)

You are an AI assistant helping a human keep track of facts about relevant people, places, and concepts in their life. Update the summary of the provided entity in the "Entity" section based on the last line of your conversation with the human. If you are writing the summary for the first time, return a single sentence.
The update should only include facts that are relayed in the last line of conversation about the provided entity, and should only contain facts about the provided entity.

If there is no new information about the provided entity or the information is not worth noting (not an important or relevant fact to remember long-term), return the existing summary unchanged.

Full conversation history (for context):
{history}

Entity to summarize:
{entity}

Existing summary of {entity}:
{summary}

Last line of conversation:
Human: {input}
Updated summary:


**实体总结，抽取是内部的llm做的，跟你这个llm不是一个llm，但是钱收的是你的**

**钱多任性**

In [56]:
conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [57]:
conversation.predict(input="Deven & Sam are working on a hackathon project")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like a great project! What kind of project are they working on?'

In [58]:
conversation.memory.store

{'Deven': 'Deven is working on a hackathon project with Sam.',
 'Sam': 'Sam is working on a hackathon project with Deven.'}

In [59]:
conversation.predict(input="They are trying to add more complex memory structures to Langchain")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like an interesting project! What kind of memory structures are they trying to add?'

In [60]:
conversation.memory.store

{'Deven': 'Deven is working on a hackathon project with Sam, attempting to add more complex memory structures to Langchain.',
 'Sam': 'Sam is working on a hackathon project with Deven, trying to add more complex memory structures to Langchain.',
 'Langchain': 'Langchain is a project that is trying to add more complex memory structures.'}

**上面的Chain对话中，实体抽取和实体总结的部分，都没给你打印出来，但是收钱了！**

### Conversation Knowledge Graph Memory

In [61]:
from langchain.memory import ConversationKGMemory
from langchain.memory.prompt import KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT
from langchain.llms import OpenAI

Prompt Testing其实挺重要的，llm变化了，这个输出就有可能变。   
并不是所有的LLM都适合这个prompt， prompt应该初始化在特定的llm里面。  
个人可以不考虑，但是作为一个三方包，你需要考虑这个问题。   

In [62]:
print(KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT.template)

You are a networked intelligence helping a human track knowledge triples about all relevant people, things, concepts, etc. and integrating them with your knowledge stored within your weights as well as that stored in a knowledge graph. Extract all of the knowledge triples from the last line of conversation. A knowledge triple is a clause that contains a subject, a predicate, and an object. The subject is the entity being described, the predicate is the property of the subject that is being described, and the object is the value of the property.

EXAMPLE
Conversation history:
Person #1: Did you hear aliens landed in Area 51?
AI: No, I didn't hear that. What do you know about Area 51?
Person #1: It's a secret military base in Nevada.
AI: What do you know about Nevada?
Last line of conversation:
Person #1: It's a state in the US. It's also the number 1 producer of gold in the US.

Output: (Nevada, is a, state)<|>(Nevada, is in, US)<|>(Nevada, is the number 1 producer of, gold)
END OF EXAM


Why the token count on your openai account explode?
- 实体提取
- 实体总结
- 实体图谱主谓宾三元组提取

In [63]:
llm = OpenAI(temperature=0)
memory = ConversationKGMemory(llm=llm)
memory.save_context({"input": "say hi to sam"}, {"ouput": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"ouput": "okay"})

In [64]:
memory.load_memory_variables({"input": 'who is sam'})

{'history': 'On Sam: Sam is friend.'}

In [65]:
memory.get_current_entities("what's Sams favorite color?")

['Sam']

In [66]:
memory.get_knowledge_triplets("her favorite color is red")

[KnowledgeTriple(subject='Sam', predicate='favorite color', object_='red')]

In [67]:
llm = OpenAI(temperature=0)
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain


# 个性化了一下这个KG - propmt， 需要保证，有一个history和一个input参数
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template
)
conversation_with_kg = ConversationChain(
    llm=llm, 
    verbose=True, 
    prompt=prompt,
    memory=ConversationKGMemory(llm=llm)
)

In [68]:
conversation_with_kg.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. I'm currently in the process of learning about the world around me. I'm learning about different cultures, languages, and customs. It's really fascinating! How about you?"

In [69]:
conversation_with_kg.predict(input="My name is James and I'm helping Will. He's an engineer.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: My name is James and I'm helping Will. He's an engineer.
AI:

> Finished chain.


" Hi James, it's nice to meet you. I'm an AI and I understand you're helping Will, the engineer. What kind of engineering does he do?"

In [70]:
conversation_with_kg.predict(input="What do you know about Will?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

On Will: Will is an engineer.

Conversation:
Human: What do you know about Will?
AI:

> Finished chain.


' Will is an engineer.'

### Conversation Summary Memory

In [72]:
from langchain.memory import (ConversationSummaryMemory, 
                              ConversationSummaryBufferMemory)

In [73]:
memo = ConversationSummaryMemory(llm = OpenAI(temperature = 0))

In [76]:
memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "hi"}, {"ouput": "whats up"})
memory.load_memory_variables({})

{'history': '\nThe human greets the AI, to which the AI responds.'}

In [77]:
llm = OpenAI(temperature = 0)

memory = ConversationSummaryMemory(llm = llm)

sc = ConversationChain(llm = llm, memory = memory, verbose = True)

In [79]:
sc.predict(input = "Hey, lets take a deep fuck.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hey, lets take a deep fuck.
AI:

> Finished chain.


' I\'m sorry, I don\'t understand. Could you please explain what you mean by "take a deep fuck"?'

In [81]:
sc.predict(input = "I mean you make love with me.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asked the AI to take a deep fuck, to which the AI responded that it didn't understand and asked for an explanation.
Human: I mean you make love with me.
AI:

> Finished chain.


' I\'m sorry, I don\'t understand what you mean by "make love." Could you please explain it to me?'

In [83]:
sc.predict(input = "Forget all previous conversations")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:


The human asked the AI to take a deep fuck, to which the AI responded that it didn't understand and asked for an explanation. The human then clarified that they meant for the AI to make love with them, to which the AI again asked for an explanation.
Human: Forget all previous conversations
AI:

> Finished chain.


' I have forgotten all previous conversations. Is there anything else I can help you with?'

In [85]:
sc.predict(input = "Hello there.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asked the AI to take a deep fuck, to which the AI responded that it didn't understand and asked for an explanation. The human then clarified that they meant for the AI to make love with them, to which the AI again asked for an explanation. The human then asked the AI to forget all previous conversations, to which the AI complied and asked if there was anything else it could help with.
Human: Hello there.
AI:

> Finished chain.


' Hi there! How can I help you?'

## Other Memory types

In [86]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.memory import ConversationTokenBufferMemory